In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import numpy as np
import random
import time
import pygame
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import dqnmodel

%aimport gui.gui

pygame 2.1.2 (SDL 2.0.16, Python 3.9.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
%run tetris-environment.ipynb

          
          
          
          
          
          
          
          
          
          
          
          
          
          
█████     
████      
████ █████
████  ████
████ █████
██████████


In [4]:
device = 'cuda'

model = dqnmodel.DQNModel(100, 'models/run-cnn-after-12000.pt')
model.to(device)

In [5]:
def get_best_state(states):
    # use the q-network (not the target network) for chosing the next state
    q_values = model.model(states)
    return torch.argmax(q_values)

In [6]:
env = TetrisEnvRendered()

In [7]:
# env.reset()
# states_to_render = []

# while True:
#     states, states_pretty, scores, clears, dones, moves = env.get_next_states()

#     chosen_index = random.choice(range(len(states))) # get_best_state(torch.from_numpy(states.reshape(-1, 1, 20, 10)).float().to(device), False)

#     states_to_render.append((states_pretty[chosen_index], moves[chosen_index], env.current_piece))

#     if dones[chosen_index]:
#         print(f'Score: {env.score}')
#         print(f'Clears: {env.clears}, t-spins: {env.tspins}, alll_clears: {env.all_clears}')
#         break
#     else:
#         env.step(states[chosen_index], states_pretty[chosen_index], clears[chosen_index], scores[chosen_index])    

In [8]:
runs = []
def render_run():
    model.model.eval()
    with torch.no_grad():
        env.reset()
        states_to_render = []

        while True:
            states, states_pretty, scores, clears, dones, moves = env.get_next_states()

            chosen_index = get_best_state(torch.from_numpy(states.reshape(-1, 1, 20, 10)).float().to(device))

            states_to_render.append((states_pretty[chosen_index], moves[chosen_index], env.current_piece, clears[chosen_index]))

            if dones[chosen_index]:
                print(f'Score: {env.score}')
                print(f'Clears: {env.clears}, t-spins: {env.tspins}, alll_clears: {env.all_clears}')
                break
            else:
                env.step(states[chosen_index], states_pretty[chosen_index], clears[chosen_index], scores[chosen_index])

        runs.append(states_to_render)
    
for i in range(3):
    render_run()

Score: 65387.5
Clears: [192, 68, 14, 17], t-spins: 9, alll_clears: 0
Score: 27087.5
Clears: [134, 24, 4, 3], t-spins: 4, alll_clears: 0
Score: 64812.5
Clears: [250, 81, 15, 6], t-spins: 10, alll_clears: 0


In [7]:
# # store run
# with open('runs/65k-good-quad-streak.pkl', 'wb') as file:
#     pickle.dump(runs[0], file, pickle.HIGHEST_PROTOCOL)

# load run
with open('runs/65k-good-quad-streak.pkl', 'rb') as file:
    run = pickle.load(file)

In [14]:
g = gui.gui.Gui(sleep=15)

last_state = np.ones((20,10)) * -1

time.sleep(5)

for state, move, piece, clears in run: #runs[0]:
    p = Piece(piece)
    
    for m in move:
        if m == 'ml':
            p.pos[1] -= 1
        if m == 'mr':
            p.pos[1] += 1
        if m == 'sd':
            p.pos[0] += 1

        if m == 'rr':
            p.rot = (p.rot + 1) % p.pdata.num_rot
        if m == 'rl':
            p.rot = (p.rot - 1) % p.pdata.num_rot
                      
        g.draw(last_state, p)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
    
    if clears[0] == 4:
        g.play_sound_quad()
    elif clears[1] and clears[0] > 0:
        g.play_sound_tspin()
    elif clears[2]:
        pass # all-clear
    elif clears[0] > 0:
        g.play_sound_clear()
    
    last_state = state

error: display Surface quit

In [ ]:
pygame.quit()